# Graphing network packets

## Preparing data

The data source comes from a publicly available network forensics repository: http://www.netresec.com/?page=PcapFiles. The selected file is https://download.netresec.com/pcap/maccdc-2012/maccdc2012_00000.pcap.gz.

```
tcpdump -qns 0 -r maccdc2012_00000.pcap | grep tcp > maccdc2012_00000.txt
```

For example, here is a snapshot of the resulting output:

```
09:30:07.780000 IP 192.168.202.68.8080 > 192.168.24.100.1038: tcp 1380
09:30:07.780000 IP 192.168.24.100.1038 > 192.168.202.68.8080: tcp 0
09:30:07.780000 IP 192.168.202.68.8080 > 192.168.24.100.1038: tcp 1380
09:30:07.780000 IP 192.168.202.68.8080 > 192.168.24.100.1038: tcp 1380
09:30:07.780000 IP 192.168.27.100.37877 > 192.168.204.45.41936: tcp 0
09:30:07.780000 IP 192.168.24.100.1038 > 192.168.202.68.8080: tcp 0
09:30:07.780000 IP 192.168.202.68.8080 > 192.168.24.100.1038: tcp 1380
09:30:07.780000 IP 192.168.202.68.8080 > 192.168.24.100.1038: tcp 1380
09:30:07.780000 IP 192.168.202.68.8080 > 192.168.24.100.1038: tcp 1380
09:30:07.780000 IP 192.168.202.68.8080 > 192.168.24.100.1038: tcp 1380
```

Given the directional nature of network traffic and the numerous ports per node, we will simplify the graph by treating traffic between nodes as undirected and ignorning the distinction between ports. The graph edges will have weights represented by the total number of bytes across both nodes in either direction.

```
python pcap_to_parquet.py maccdc2012_00000.txt
```

The resulting output will be two Parquet dataframes, `maccdc2012_nodes.parq` and `maccdc2012_edges.parq`.

## Loading data

In [ ]:
import holoviews as hv
import dask.dataframe as dd
from holoviews.operation.datashader import (
    datashade, aggregate, dynspread,
    bundle_graph, split_dataframe, regrid, stack
)
from holoviews.element.graphs import layout_nodes
from datashader.layout import forceatlas2_layout, random_layout

hv.extension('bokeh')

%opts RGB Graph Nodes [bgcolor='black' width=800 height=800 xaxis=None yaxis=None]

In [ ]:
edges_df = dd.read_parquet('./data/maccdc2012_full_edges.parq').compute()
edges_df = edges_df.reset_index(drop=True)
len(edges_df)

## Edge bundling & layouts

Datashader and HoloViews provide support for a number of different graph layouts including circular, force atlas and random layouts. Since large graphs with thousands of edges can become quite messy when plotted datashader also provides functionality to bundle the edges.

#### Circular layout

By default the HoloViews Graph object lays out nodes using a circular layout. Once we have declared the ``Graph`` object easily apply the ``bundle_graph`` operation, setting ``split=False`` to stop the operation from splitting the edges into individual arrays. This allows datashader to operate more quickly on the resulting array. We also overlay the datashaded graph with the nodes, letting us identify each node by hovering.

In [ ]:
%%opts Nodes (size=5)
circular = bundle_graph(hv.Graph(edges_df), split=False)
pad = dict(x=(-1.2, 1.2), y=(-1.2, 1.2))
datashade(circular, width=800, height=800) * circular.nodes.redim.range(**pad)

#### Force Atlas 2 layout

For other graph layouts you can use the ``layout_nodes`` operation supplying the datashader or NetworkX layout function.

In [ ]:
%%opts Nodes (size=5)
graph = layout_nodes(hv.Graph(edges_df), layout=forceatlas2_layout)
forceatlas = bundle_graph(graph, split=False)
pad = dict(x=(-.5, 1.3), y=(-.5, 1.3))
datashade(forceatlas, width=800, height=800) * forceatlas.nodes.redim.range(**pad)

#### Random layout

In [ ]:
%%opts Nodes (size=5)
graph = layout_nodes(hv.Graph(edges_df), layout=random_layout)
random = bundle_graph(graph, split=False)
pad = dict(x=(-.05, 1.05), y=(-0.05, 1.05))
datashade(random, width=800, height=800) * random.nodes.redim.range(**pad)

## Showing nodes with active traffic

To select just nodes with active traffic we will split the dataframe of bundled paths and then apply ``select`` on the new Graph to select just those edges with a weight of more than 10,000. By overlaying the sub-graph of high traffic edges we can take advantage of the interactive hover and tap features that bokeh provides while still revealing the full datashaded graph in the background.

In [ ]:
def split_edges(graph):
    paths = split_dataframe(graph.edgepaths.data[0])
    return graph.clone((graph.data, graph.nodes, paths))

def join_edges(graph):
    return graph.clone((graph.data, graph.nodes, [graph.edgepaths.array()]))

In [ ]:
%%opts Graph (edge_line_color='white' edge_hover_line_color='blue')
split_circular = split_edges(circular)
pad = dict(x=(-1.2, 1.2), y=(-1.2, 1.2))
datashade(circular, width=800, height=800) * split_circular.select(weight=(10000, None)).redim.range(**pad)

## Highlight TCP and UDP traffic

Using the same selection features we can highlight TCP and UDP connections separately again by overlaying it on top of the full datashaded graph. The edges can be revealed over the highlighted nodes and by setting an alpha level we can also reveal connections with both TCP (blue) and UDP (red) connections in purple.

In [ ]:
%%opts Graph (edge_alpha=0 edge_hover_alpha=0.5 edge_nonselection_alpha=0 node_size=8 node_alpha=0.5)
split_atlas = split_edges(forceatlas)
udp_style = dict(edge_hover_line_color='red', node_fill_color='red', edge_selection_line_color='red')
tcp_style = dict(edge_hover_line_color='blue', node_fill_color='blue', edge_selection_line_color='blue')
udp = split_atlas.select(protocol='udp', weight=(1000, None)).opts(style=udp_style)
tcp = split_atlas.select(protocol='tcp', weight=(1000, None)).opts(style=tcp_style)
datashade(forceatlas, width=800, height=800, normalization='log', cmap=['black', 'white']) * tcp * udp

### Combining datashaded plots

In [ ]:
from bokeh.palettes import Blues9,  Reds9
ranges = dict(x_range=(-.5, 1.6), y_range=(-.5, 1.6), width=800, height=800)
tcp, udp = (join_edges(split_atlas.select(protocol=p)) for p in ['tcp', 'udp'])
stack(datashade(tcp, cmap=Blues9, **ranges) * datashade(udp, cmap=Reds9, **ranges) *
      dynspread(datashade(forceatlas.nodes, cmap=['white'], **ranges)), link_inputs=True) 